# Investigate problems related to work not_work data

## Gather data

In [412]:
import constants
import pandas as pd
import os
from IPython.display import display
import json
import numpy as np
path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))
input_directory = parent + '/'+constants.output_data_segment_keyword_matrix
segment_df = pd.read_csv(input_directory + constants.output_segment_keyword_matrix_document_index)
int_codes = segment_df['IntCode'].to_list()
with open(parent+'/'+constants.output_data_segment_keyword_matrix + "metadata_partitions.json") as read_file:
        metadata_partitions = json.load(read_file)
input_folder = parent+'/'+constants.input_data
input_file = constants.input_files_biodata_birkenau
df_biodata = pd.read_csv(input_folder+input_file)
df_biodata = df_biodata[df_biodata.IntCode.isin(int_codes)]

Load the very original segment data

In [413]:
input_directory = constants.input_data
input_files = constants.input_files_segments
input_files = [parent+'/'+input_directory+i for i in input_files]
df = pd.concat([pd.read_csv(el) for el in input_files])
df = df.reset_index()

Load the segment keyword matrix

In [414]:
 # Load the input data
input_directory = parent +'/'+constants.output_data_segment_keyword_matrix
# Read the segment index term matrix
data = np.load(input_directory + constants.output_segment_keyword_matrix_data_file.replace('.txt', '.npy'),  allow_pickle=True)
# Read the column index (index terms) of the matrix above
features_df = pd.read_csv(input_directory + constants.output_segment_keyword_matrix_feature_index)

## Check the original data

Get the int code of those who worked and who did not work

Not work first

In [415]:
int_codes_notwork = []
for element in metadata_partitions['notwork']:
    int_codes_notwork.append(int_codes[element])
    
    

Choose randomly from the list

In [416]:
import random
not_work_random = random.sample(int_codes_notwork,100)

Check if contains work

In [417]:
#df[df.IntCode==not_work_random].KeywordLabel.value_counts()
for element in not_work_random:
    random_df = df[df.IntCode==element]
    if random_df[random_df['KeywordLabel'].str.contains('labor')].size >0:
        print (element)
        print (random_df[random_df['KeywordLabel'].str.contains('labor')]['KeywordLabel'])


10351
239045    camp forced labor
Name: KeywordLabel, dtype: object
3716
101155    German forced labor civilian supervisors
Name: KeywordLabel, dtype: object
54081
1024562    camp forced labor
1024607    camp forced labor
Name: KeywordLabel, dtype: object
35130
701836    camp forced labor
Name: KeywordLabel, dtype: object
11031
252898    camp forced labor
Name: KeywordLabel, dtype: object


Work second

In [418]:
int_codes_work = []
for element in metadata_partitions['work']:
    int_codes_work.append(int_codes[element])
    
    

Choose randomly from the list

In [419]:
import random
work_random = random.sample(int_codes_work,100)

Check if contains work

In [420]:
#df[df.IntCode==not_work_random].KeywordLabel.value_counts()
for element in work_random:
    random_df = df[df.IntCode==element]
    if random_df[random_df['KeywordLabel'].str.contains('labor')].size >0:
        print (element)
        print (random_df[random_df['KeywordLabel'].str.contains('labor')]['KeywordLabel'])


253
6738                 camp forced labor
6740    personal services forced labor
6749                 camp forced labor
6751    personal services forced labor
6785                 camp forced labor
6787    personal services forced labor
6813                 camp forced labor
6815    personal services forced labor
Name: KeywordLabel, dtype: object
10587
244006      camp forced labor
244009    quarry forced labor
Name: KeywordLabel, dtype: object
24366
523131        camp forced labor
523134    platform forced labor
Name: KeywordLabel, dtype: object
48660
926528            camp forced labor
926529    construction forced labor
Name: KeywordLabel, dtype: object
19230
420570                        camp forced labor
420571                  forced labor conditions
420572    mobile construction crew forced labor
420575                      quarry forced labor
420595                        camp forced labor
420596             corpse disposal forced labor
420602                        camp force

2355
65493                    camp forced labor
65496    textiles and garment forced labor
Name: KeywordLabel, dtype: object
40163
783992      camp forced labor
783996    quarry forced labor
Name: KeywordLabel, dtype: object
47991
910134                camp forced labor
910135    camp maintenance forced labor
Name: KeywordLabel, dtype: object
11287
258026               camp forced labor
258028    corpse disposal forced labor
258126               camp forced labor
258129             quarry forced labor
Name: KeywordLabel, dtype: object
25642
547051               camp forced labor
547052    corpse disposal forced labor
Name: KeywordLabel, dtype: object
55036
1053068                         camp forced labor
1053108                         camp forced labor
1053137    personal property sorting forced labor
1053180                         camp forced labor
Name: KeywordLabel, dtype: object
53025
1003904                         camp forced labor
1003906    personal property sorting forced l

## Check the segment keyword matrix

In [421]:
index_social_bond = features_df.KeywordLabel.to_list().index('social bonds')

First not work

In [422]:
seg_keyword = np.take(data,metadata_partitions['notwork'])

In [423]:
assert len(seg_keyword) == len(int_codes_notwork)

Check how many times a person mentions social bonds on average

In [424]:
counts = []
for element in seg_keyword:
    counts.append(np.nonzero(element[:,index_social_bond])[0].shape[0])
df_counts = pd.DataFrame(counts)

Percentage of those who mention it at least once

In [425]:
len(df_counts[df_counts[0]>0])/len(df_counts)*100

14.968279143536876

In [426]:
len(df_counts[df_counts[0]>0])

755

In [427]:
percentage_mentioning_at_least_one_count_notwork=len(df_counts[df_counts[0]>0])/len(df_counts)*100

Total number of mentioning social bonds

In [428]:
df_counts[0].sum()

1013

In [429]:
total_n_mentioning_social_bonds_notwork = df_counts[0].sum()

Then check work

In [430]:
seg_keyword = np.take(data,metadata_partitions['work'])

In [431]:
assert len(seg_keyword) == len(int_codes_work)

Check how many times a person mentions social bonds on average

In [432]:
counts = []
for element in seg_keyword:
    counts.append(np.nonzero(element[:,index_social_bond])[0].shape[0])
df_counts = pd.DataFrame(counts)

In [433]:
df_counts.describe()

,0
count,1584.000000
mean,0.455808
std,0.965909
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,10.000000


Percentage of those who mention it at least once

In [434]:
percentage_mentioning_at_least_one_count_work=len(df_counts[df_counts[0]>0])/len(df_counts)*100

In [435]:
len(df_counts[df_counts[0]>0])

434

Total number of mentioning social bonds

In [436]:
df_counts[0].sum()

722

In [437]:
total_n_mentioning_social_bonds_work = df_counts[0].sum()

## Check the markov training

In [438]:
from markov_modelling import markov_utils as mu

First those who worked

In [439]:
seg_keyword = np.take(data,metadata_partitions['work'])
trajs = mu.estimate_fuzzy_trajectories(seg_keyword)

Check how many times social bonds occur

In [440]:
n=0
row_occurrences_work = 0
for traj in trajs:
    if index_social_bond in traj:
        n+=1
        occurrences = np.count_nonzero(traj == index_social_bond)
        row_occurrences_work = row_occurrences_work+occurrences

    
    

Total number of trajectories where social bonds mentioned at least once  

In [441]:
print (n/len(trajs))

0.18904040404040404


In [442]:
percentage_traj_mentionining_once_to_total_work = (n/len(trajs))*100

In [443]:
n/len(trajs)

0.18904040404040404

Estimate the Markov model1

In [444]:
msm_work = mu.estimate_markov_model_from_trajectories(trajs)

Then those who did not work

In [445]:
seg_keyword = np.take(data,metadata_partitions['notwork'])
trajs = mu.estimate_fuzzy_trajectories(seg_keyword)
msm_notwork = mu.estimate_markov_model_from_trajectories(trajs)

Check how many times social bonds occur

In [446]:
n=0
row_occurrences_notwork = 0
for traj in trajs:
    if index_social_bond in traj:
        n+=1
        occurrences = np.count_nonzero(traj == index_social_bond)
        row_occurrences_notwork = row_occurrences_notwork+occurrences

    

Total number of trajectories where social bonds mentioned at least once

In [447]:
print (n)

13047


In [448]:
print (n/len(trajs))

0.10346550356859635


In [449]:
percentage_traj_mentionining_once_to_total_notwork = n/len(trajs)*100

In [450]:
ratio_mentioning_trajs_total_trajs_not_work = n/len(trajs)

In [451]:
msm_notwork.pi.shape

(110,)

In [452]:
msm_work.pi.shape

(111,)

In [453]:
index_social_bond

97

Compare stationary probabilities

In [454]:
pi_social_bonds_work=msm_work.pi[msm_work._full2active[index_social_bond]]

In [455]:
pi_social_bonds_notwork=msm_notwork.pi[msm_notwork._full2active[index_social_bond]]

# Problem

- 27% of the working people mention social bonds at least once
- 15% of the non working people mention social bonds at least once
- this explains why the Fisher test tells that 'social bonds' is more associated with the working people
- 10.3% of the trajectories of those who did not work include 'social bonds' at least once
- 18.8% of the trajectories of those who worked include 'social bonds' at least once
- The ratios are more or less the same
 
**BUT**

- but when stationary probabilities are compared, the stationary probability of social bonds in the non working is higher than the stationary prob of work

In [461]:
percentage_mentioning_at_least_one_count_work

27.3989898989899

In [462]:
percentage_mentioning_at_least_one_count_notwork

14.968279143536876

In [463]:
percentage_traj_mentionining_once_to_total_notwork

10.346550356859636

In [464]:
percentage_traj_mentionining_once_to_total_work

18.904040404040405

In [465]:
percentage_mentioning_at_least_one_count_work > percentage_mentioning_at_least_one_count_notwork


True

In [466]:
percentage_traj_mentionining_once_to_total_work > percentage_traj_mentionining_once_to_total_notwork

True

In [467]:
percentage_mentioning_at_least_one_count_work / percentage_mentioning_at_least_one_count_notwork

1.8304702655696032

In [468]:
percentage_traj_mentionining_once_to_total_work / percentage_traj_mentionining_once_to_total_notwork

1.8270862994937496

In [471]:
pi_social_bonds_notwork > pi_social_bonds_work

True

In [472]:
pi_social_bonds_notwork / pi_social_bonds_work

1.0795553635776463